In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import ast

In [2]:
users_final = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/users_final_numeric.csv')
products_final= pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/products_final_numeric.csv')

### Baseline 2-tower model

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dot, BatchNormalization
from tensorflow.keras.models import Model

Sampling for batch size equality

In [4]:
user_input = Input(shape=(16,), name='user_input')
item_input = Input(shape=(31,), name='item_input')

In [5]:
#Changed from baseline
user_tower = Dense(128, activation='relu')(user_input)
user_tower = BatchNormalization()(user_tower)

In [6]:
item_tower = Dense(128, activation='relu')(item_input)
item_tower = BatchNormalization()(item_tower)

In [7]:
dot_product = Dot(axes=1)([user_tower, item_tower])

In [8]:
model = Model(inputs=[user_input, item_input], outputs=dot_product)

In [9]:
model.compile(optimizer='adam', loss='mse')

In [10]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 31)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      2,176 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      4,096 │ item_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128)       │        512 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,296 (28.50 KB)

 Trainable params: 6,784 (26.50 KB)

 Non-trainable params: 512 (2.00 KB)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def train_model(users_train, products_train, model, batch_size, num_epochs):
    users_train, users_val = train_test_split(users_train, test_size=0.1, random_state=42)
    products_train, products_val = train_test_split(products_train, test_size=0.1, random_state=42)

    for epoch in range(num_epochs):
        # Generate random batches
        user_indices = np.random.randint(0, len(users_train), size=batch_size)
        product_indices = np.random.randint(0, len(products_train), size=batch_size)

        # Balanced sampling
        users_final['interaction'] = users_final['product_id'].isin(products_final['product_id']).astype(int)

        positive_samples = users_train[users_train['interaction'] == 1].sample(batch_size // 2, random_state=42)
        negative_samples = users_train[users_train['interaction'] == 0].sample(batch_size // 2, random_state=42)
        user_data_batch = pd.concat([positive_samples, negative_samples])

        product_data = products_train.iloc[product_indices]

        # Create similarity labels
        target_similarity = (users_train.iloc[user_indices]['product_id'].values ==
                             products_train.iloc[product_indices]['product_id'].values).astype(int)

        # Drop unnecessary columns
        user_data = user_data_batch.drop(columns=['product_id'], errors='ignore')
        product_data = product_data.drop(columns=['product_id', 'name_embedding'], errors='ignore')

        # Train the model
        model.fit([user_data.values, product_data.values], target_similarity, epochs=1, batch_size=batch_size)

        # Validation at the end of each epoch
        user_data_val = users_val.drop(columns=['product_id'], errors='ignore')
        product_data_val = products_val.drop(columns=['product_id', 'name_embedding'], errors='ignore')

        val_target_similarity = (users_val['product_id'].values == products_val['product_id'].values).astype(int)
        val_predictions = model.predict([user_data_val.values, product_data_val.values])

        val_auc = roc_auc_score(val_target_similarity, val_predictions)
        print(f"Epoch {epoch + 1}/{num_epochs} - Validation AUC: {val_auc:.4f}")

#### Evaluate the Model

In [12]:
def evaluate_model(users_test, products_test, model, batch_size):
    user_indices_test = np.random.randint(0, len(users_test), size=batch_size)
    product_indices_test = np.random.randint(0, len(products_test), size=batch_size)

    user_data_test = users_test.iloc[user_indices_test].drop(columns=['product_id'], errors='ignore')
    product_data_test = products_test.iloc[product_indices_test].drop(columns=['product_id', 'name_embedding'], errors='ignore')

    target_similarity_test = (users_test.iloc[user_indices_test]['product_id'].values ==
                              products_test.iloc[product_indices_test]['product_id'].values).astype(int)

    predicted_similarity = model.predict([user_data_test.values, product_data_test.values])

    threshold = 0.5
    predicted_labels = (predicted_similarity >= threshold).astype(int)

    auc_score = roc_auc_score(target_similarity_test, predicted_similarity)
    precision = precision_score(target_similarity_test, predicted_labels)
    recall = recall_score(target_similarity_test, predicted_labels)

    print(f"Test AUC: {auc_score:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

### Generating Recommendations

In [13]:

def generate_recommendations(users_df, products_df, model, user_id=None):
    if user_id is None:
        user_id = np.random.choice(users_df['user_id'].unique())

    print(f"Generating top 10 recommendations for user {user_id}...")

    # Extract data for the selected user
    user_data_given = users_df[users_df['user_id'] == user_id].drop(columns=['product_id'], errors='ignore')

    # Precompute product embeddings
    product_data = products_df.drop(columns=['product_id', 'name_embedding'], errors='ignore')
    precomputed_product_embeddings = model.predict(product_data.values)

    # Generate recommendations
    user_embedding = model.predict(user_data_given.values)
    similarity_scores = np.dot(precomputed_product_embeddings, user_embedding.T)

    sorted_indices = np.argsort(similarity_scores.flatten())[::-1]
    top_10_recommendations = products_df.iloc[sorted_indices[:10]]

    print("Top 10 Recommendations:")
    print(top_10_recommendations[['product_id', 'name']])

#### Main function

In [14]:
def main(users_df, products_df, model, batch_size, num_epochs):
    # Split data into train and test sets
    users_train, users_test = train_test_split(users_df, test_size=0.2, random_state=42)
    products_train, products_test = train_test_split(products_df, test_size=0.2, random_state=42)

    print("Starting training...")
    train_model(users_train, products_train, model, batch_size, num_epochs)

    print("Evaluating model...")
    evaluate_model(users_test, products_test, model, batch_size)

    print("Generating recommendations...")
    generate_recommendations(users_df, products_df, model)

In [15]:
# Example usage
batch_size = 500
num_epochs = 25
main(users_final, products_final, model, batch_size, num_epochs)

Starting training...


KeyError: 'interaction'